<a href="https://colab.research.google.com/github/Margo-Kim/Hate_Detection_G/blob/main/df_to_postgreSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install psycopg2

In [2]:
import psycopg2 as ps
import pandas as pd

In [3]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving youtube_comment.csv to youtube_comment.csv


In [4]:
df = pd.read_csv('youtube_comment.csv', index_col = 0)
df.head()

,comment,likes
0,다들 자기편 들어줄거로 알았는데 여론 개박살나니 커뮤 막아버리는 우리펄 폼 미쳤다ㄷㄷ,8322.0
1,녹음기로 말 많은데 녹음기 전에 3살많은 애가 초2상대로 교실에서 성범죄 저지르고도...,7283.0
2,엄마라는 사람도 웹툰에 감정기복으로 아이<br>가르칠 자신 없다고 그려 놨더니 남의...,7159.0
3,감사함을 잊는건 봤어도,NaN
<br>감사함을 원수로 갚는거는 진짜 살다살다 처음봄,6788,NaN


In [5]:
host_name = 'database-1.c1zguc0w15aj.us-east-2.rds.amazonaws.com'
dbname = 'postgres'
port = '5432'
username = 'postgres'
password = 'Podonglove!2'
conn = None

In [6]:
def connect_to_db(host_name, dbname, port, username, password):
  try:
    conn = ps.connect(host=host_name, database = dbname, user=username, password=password, port=port)

  except ps.OperationalError as e:
    raise e
  else:
    print('Connected!')
    return conn

conn = connect_to_db(host_name, dbname, port, username, password)

Connected!


In [ ]:
# create DB table

def create_table(curr):
  create_table_command = (""" CREATE TABLE IF NOT EXISTS videos (
                          comment TEXT NOT NULL,
                          likes INTEGER NOT NULL)""")

  curr.execute(create_table_command)

In [38]:
# connection to the database and cursor --> run SQL and fetch result from the DB

curr = conn.cursor()

#create_table(curr)

In [ ]:
conn.commit()

In [39]:
# Step 1. check to see if video exists

def update_db(curr,df):
  temp_df = pd.DataFrame(columns = ['comment', 'likes'])

  for i, row in df.iterrows():
    if check_if_video_exists(curr, row['comment']): #If the video already exists then will update
      update_row(curr,row['comment'], row['likes'])
    else:
      temp_df = pd.concat([temp_df, pd.DataFrame([row])]).reset_index(drop = True) #The video doesn't exists so we will append to the db table

  return temp_df


In [40]:
# check if video exists
def check_if_video_exists(curr, comment):
    query = ("""SELECT comment FROM videos WHERE comment = %s""")
    curr.execute(query, (str(comment),)) #have to pass in tuple

    return curr.fetchone() is not None #fetchone -> return a single row from the table


In [41]:
# update row if video exists

def update_row(curr, comment, likes):
  query = (""" UPDATE videos
              SET likes = %s
              WHERE comment =%s; """)

  vars_to_update = (likes, comment)
  curr.execute(query, vars_to_update)

In [42]:
new_vid_df = update_db(curr,df)

In [43]:
conn.commit()

In [45]:
new_vid_df

,comment,likes
0,다들 자기편 들어줄거로 알았는데 여론 개박살나니 커뮤 막아버리는 우리펄 폼 미쳤다ㄷㄷ,8322.0
1,녹음기로 말 많은데 녹음기 전에 3살많은 애가 초2상대로 교실에서 성범죄 저지르고도...,7283.0
2,엄마라는 사람도 웹툰에 감정기복으로 아이<br>가르칠 자신 없다고 그려 놨더니 남의...,7159.0
3,감사함을 잊는건 봤어도,NaN
4,6788,NaN
...,...,...
103,본인 자식이 귀하면 남의 자식도 귀하다,478.0
104,능력에 비해 너무 잘됐는데 지옥 끝까지 떨어지는구나 ㅋㅋㅋㅋㅋ<br>침착맨도 짠하네...,476.0
105,녹음기 넣은 것처럼 조용히 커뮤글 pdf따고 있는 주호민이면 개추ㅋㅋ,474.0
106,아주 떳떳하고 당당하신거 보니 나중에 49일간 7번의 재판을 무사히 통과할 자신이 ...,469.0


In [56]:
# write insert command

def insert_into_table(curr, comment, likes):
  try:
    likes = int(likes)
  except ValueError:
    likes = 0

  insert_into_videos = ("""INSERT INTO videos (comment, likes)
                          VALUES (%s, %s); """)

  row_to_insert = (comment, likes)

  curr.execute(insert_into_videos, row_to_insert)

In [57]:
def append_from_df_to_db(curr, df):
  for i, row in df.iterrows():
    insert_into_table(curr, row['comment'], row['likes'])

In [58]:
append_from_df_to_db(curr, new_vid_df)

In [59]:
conn.commit()